In [ ]:
import tensorflow as tf
from sklearn.datasets import load_boston
from sklearn.preprocessing import scale
import numpy as np


In [ ]:
learning_rate = 0.01
epochs = 10000
display_epoch = epochs//20
n_train = 300
n_valid = 100


In [ ]:
features, prices = load_boston(True)
n_test = len(features) - n_train - n_valid

# Keep n_train samples for training
train_features = tf.cast(scale(features[:n_train]), dtype=tf.float32) #
train_prices = prices[:n_train]

# Keep n_valid samples for validation
valid_features = tf.cast(scale(features[n_train:n_train+n_valid]), dtype=tf.float32)
valid_prices = prices[n_train:n_train+n_valid]

# Keep remaining n_test data points as test set
test_features = tf.cast(scale(features[n_train+n_valid:n_train+n_valid+n_test]), dtype=tf.float32)
test_prices = prices[n_train + n_valid : n_train + n_valid + n_test]



In [ ]:
# functio returning the predicted value of y
def prediction(x, weights, bias):
  return tf.add(tf.matmul(x,weights), bias) # our predicted (learned)  m and c, expression is like y = m*x + c


In [ ]:
# A loss function using root mean-squared error
def loss(x, y, weights, bias):
  error = prediction(x, weights, bias) - y #  how 'wrong' our predicted (learned)  y is
  squared_error = tf.square(error)
  return tf.sqrt(tf.reduce_mean(input_tensor=squared_error)) # squre root of overall mean of squared error.

In [ ]:
# Find the derivative of loss with respect to weight and bias
def gradient(x, y, weights, bias):
  with tf.GradientTape() as tape:
    loss_value = loss(x, y, weights, bias)
  return tape.gradient(loss_value, [weights, bias])# direction and value of the gradient of our weight and bias

In [ ]:
# accuracy = Number of true predictions/Total predictions
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        #print("Accuracy:", accuracy.eval({X: X_test, y: y_test}))
def accuracy(y_true,y_predicted):
    return tf.sqrt(tf.reduce_mean(input_tensor=tf.square(y_predicted-y_true)))

In [ ]:
# Start with random values for W and B on the same batch of data
W = tf.Variable(tf.random.normal([13, 1],mean=0.0, stddev=1.0, dtype=tf.float32))
B = tf.Variable(tf.zeros(1) , dtype = tf.float32)
print(W,B)
print("Initial loss: {:.3f}".format(loss(train_features, train_prices,W, B)))

In [ ]:
for e in range(epochs): #iterate for each training epoch
    deltaW, deltaB = gradient(train_features, train_prices, W, B) # direction (sign)  and value of the gradient of our weight and bias
    change_W = deltaW * learning_rate # adjustment amount for weight
    change_B = deltaB * learning_rate # adjustment amount for bias
    W.assign_sub(change_W) # subract from W
    B.assign_sub(change_B) # subract from B
    if e==0 or e % display_epoch == 0:
        # print(deltaW.numpy(), deltaB.numpy()) # uncomment if you want to see the gradients
        print("Validation loss after epoch {:02d}: {:.3f}".format(e, loss(valid_features, valid_prices, W, B)))

In [ ]:
print("Final validation loss: {:.3f}".format(loss(train_features, train_prices, W, B)))
print("Final test loss: {:.3f}".format(loss(test_features, test_prices, W, B)))
print("W = {}, B = {}".format(W.numpy(), B.numpy()))


#### example house

In [ ]:
example_house = 69
y = test_prices[example_house]
y_pred = prediction(test_features,W.numpy(),B.numpy())[example_house]
print("Actual median house value",y," in $10K")
print("Predicted median house value ",y_pred.numpy()," in $10K")